In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
sys.path.insert(1, '/home/nicolas/code/src')
sys.path.insert(1, '/home/nicolas/code/data')

from RBM import RBM
from scipy.integrate import simps
from scipy import stats


device = torch.device("cuda")
dtype = torch.float
torch.set_num_threads(4)

In [2]:
data = np.genfromtxt('../data/data_1d2c_bal_seed14.dat')
data = torch.tensor((data+1)/2, device = device, dtype = dtype)


In [3]:
data.shape

torch.Size([6000, 1000])

In [4]:

lr = 0.01
NGibbs = 100
annSteps = 0
mb_s = 600
num_pcd = 100
Nh = 20
Nv = data.shape[1]
ep_max = 10
w_hat = torch.linspace(0,1,steps=100)
_, _, V = torch.svd(data)
V = V[:,0]
N = 20000
it_mean = 50

In [5]:
myRBM = RBM(num_visible=Nv,
            num_hidden=Nh,
            device=device,
            lr=lr,
            gibbs_steps=NGibbs,
            UpdCentered=False,
            mb_s=mb_s,
            num_pcd=num_pcd,
            w_hat = w_hat,
            N = N,
            it_mean = it_mean,
            V = V,
            TMCLearning = True
           )

In [ ]:
myRBM.fit(data.T, ep_max)

IT  0
17.283262014389038
17.560485363006592
17.370328903198242
17.07879137992859
17.481587648391724
17.325709342956543
17.923680543899536
17.747706413269043
17.542551279067993
17.23416256904602
IT  1
17.157101154327393
17.145751237869263
17.411704540252686
17.359182119369507
17.057685136795044
17.527804136276245
17.386096954345703
18.48849320411682


In [ ]:
%debug

In [ ]:
vinit = torch.bernoulli(torch.rand(
        (myRBM.Nv, 1000), device=myRBM.device, dtype=myRBM.dtype))
si, _, _, _ = myRBM.Sampling(vinit, it_mcmc=NGibbs)


In [ ]:
proj_gen = torch.mv(si, V)
plt.hist(proj_gen.cpu().numpy(), bins = 100);

In [ ]:
proj_data = torch.mv(data, V)
plt.hist(proj_data.cpu().numpy(), bins = 100);